# 10_04: Modeling COVID-19 data

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd

import plotly.express as px

In [ ]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
covid19 = pd.read_csv('covid19.csv.gz', parse_dates=['date'], dtype_backend='pyarrow')

In [ ]:
covid19.info()

In [ ]:
covid19['deaths_per_million'] = covid19.total_deaths / (covid19.population / 1.0e6)
covid19['excess_per_million'] = covid19.total_excess / (covid19.population / 1.0e6)

final = covid19.groupby('country').last()

In [ ]:
final.deaths_per_million.mean(), final.deaths_per_million.std()

In [ ]:
px.histogram(final.deaths_per_million)

In [ ]:
explanatory = ['population', 'gdp_per_capita', 'population_density', 'life_expectancy',
               'median_age', 'extreme_poverty', 'human_development_index',
               'hospital_beds_per_thousand', 'percent_fully_vaccinated']

In [ ]:
fit = smf.ols('deaths_per_million ~ gdp_per_capita + human_development_index',
              data=final.reset_index()).fit()
np.sqrt(fit.mse_resid), fit.rsquared, fit.fvalue

In [ ]:
fit = smf.ols('deaths_per_million ~ extreme_poverty + percent_fully_vaccinated',
              data=final.reset_index()).fit()
np.sqrt(fit.mse_resid), fit.rsquared, fit.fvalue

In [ ]:
import itertools
list(itertools.combinations(explanatory, 2))

In [ ]:
def getfit(final, expvars):
    return smf.ols('deaths_per_million ~ ' + '+'.join(expvars), data=final.reset_index()).fit()

In [ ]:
getfit(final, ['gdp_per_capita']).fvalue

In [ ]:
fvalues = {expvars: getfit(final, expvars).fvalue for nvars in range(1, len(explanatory) + 1)
                                                  for expvars in itertools.combinations(explanatory, nvars)}

In [ ]:
bestvars = max(fvalues.keys(), key=fvalues.get)
bestvars

In [ ]:
getfit(final, bestvars).summary2()

In [ ]:
covid19['year'] = covid19.date.dt.year

In [ ]:
def getyear(year):
    return covid19[covid19.year == year].groupby('country').last()

In [ ]:
for year in range(2020, 2024):
    final_by_year = getyear(year)
    
    fvalues = {expvars: getfit(final_by_year, expvars).fvalue
               for nvars in range(1, len(explanatory) + 1)
               for expvars in itertools.combinations(explanatory, nvars)}

    bestvars = max(fvalues.keys(), key=fvalues.get)
    bestfit = getfit(final_by_year, bestvars)
    
    print(f'In {year}, the best model is {'+'.join(bestvars)} with f={bestfit.fvalue:.1f}, res={np.sqrt(bestfit.mse_resid):.1f}')

In [ ]:
getfit(getyear(2020), ['human_development_index']).summary2()

In [ ]:
fit = smf.ols('excess_per_million ~ human_development_index', data=getyear(2020).reset_index()).fit()
fit.summary2()